In [ ]:
o = __name__ == '__main__'
from pandas import Series, Index, DataFrame
from toolz.curried import *
if o:
    %reload_ext importable
    from base import BaseIndex
else:
    from .base import BaseIndex
    
from requests import *

__all__ = 'RequestIndex', 'ResponseIndex'

In [8]:
import pickle
_cache = dict()

class RequestIndex(BaseIndex):
    _base = str
    def get(self, **kwargs):
        cache = _cache is not None and memoize(cache=_cache, key=lambda a, k: pickle.dumps((a,k))) or identity
        return ResponseIndex([cache(get)(value, **kwargs) for value in self])
    
    def __add__(self, value, prefix=""):
        if not isinstance(value, str) and isiterable(value):
            return type(self)(str(_0)+prefix+str(_1) for _0, _1 in zip(self, value))
        return self.map(flip(add)(prefix+str(value)))
    
    def __truediv__(self, value):
        return self.__add__(value, '/')

class ResponseIndex(BaseIndex):
    _base = Response
    _attr_mapper = [tuple(Response.__dict__.keys()), Series], [tuple(Response.__attrs__), Index]